# 01_0_Directed_Search_Baseline

**File Description**
This file conducts a directed search across policies given a baseline scenarios. The baseline scenario is defined as the average of the range for each lever.

**Set up run parameters, import key packages, define model**

In [1]:
pf_id = 2
case = 0 #baseline
nfe = 6500
n_scenarios = 1000

In [2]:
from ema_workbench import (ScalarOutcome, Scenario, MultiprocessingEvaluator, SequentialEvaluator)
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress)
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.util import ema_logging
import pandas as pd
import numpy as np
ema_logging.log_to_stderr(ema_logging.INFO)

model, planning_steps = get_model_for_problem_formulation(pf_id)

**Define scenarios**

In [4]:
#base scenario; it's the average of the range of each uncertainty
scenarios = [Scenario('baseline', **{'discount rate 0':3,
                                       'discount rate 1':3,
                                       'discount rate 2':3,
                                       'A.1_Bmax' : 190,
                                       'A.1_Brate':1.5,
                                       'A.1_pfail':0.5,
                                       'A.2_Bmax': 190,
                                       'A.2_Brate':1.5,
                                       'A.2_pfail':0.5,
                                       'A.3_Bmax' : 190,
                                       'A.3_Brate':1.5,
                                       'A.3_pfail':0.5,
                                       'A.4_Bmax':190,
                                       'A.4_Brate':1.5,
                                       'A.4_pfail':0.5,
                                       'A.5_Bmax' : 190,
                                       'A.5_Brate':1.5,
                                       'A.5_pfail':0.5,
                                       'A.0_ID flood wave shape' :66}),
             Scenario('worstcase', **{'discount rate 0':4.5,
                                       'discount rate 1':4.5,
                                       'discount rate 2':4.5,
                                       'A.1_Bmax' : 350,
                                       'A.1_Brate':10,
                                       'A.1_pfail':1,
                                       'A.2_Bmax': 350,
                                       'A.2_Brate':10,
                                       'A.2_pfail':1,
                                       'A.3_Bmax' : 350,
                                       'A.3_Brate':10,
                                       'A.3_pfail':1,
                                       'A.4_Bmax':350,
                                       'A.4_Brate':10,
                                       'A.4_pfail':1,
                                       'A.5_Bmax' : 350,
                                       'A.5_Brate':10,
                                       'A.5_pfail':1,
                                       'A.0_ID flood wave shape' :132}),
             Scenario('bestcase', **{'discount rate 0':1.5,
                                      'discount rate 1':1.5,
                                      'discount rate 2':1.5,
                                      'A.1_Bmax' :30,
                                      'A.1_Brate':1,
                                      'A.1_pfail':0,
                                      'A.2_Bmax':30,
                                      'A.2_Brate':1,
                                      'A.2_pfail':0,
                                      'A.3_Bmax' :30,
                                      'A.3_Brate':1,
                                      'A.3_pfail':0,
                                      'A.4_Bmax':30,
                                      'A.4_Brate':1,
                                      'A.4_pfail':0,
                                      'A.5_Bmax' :30,
                                      'A.5_Brate':1,
                                      'A.5_pfail':0,
                                      'A.0_ID flood wave shape' :0})]

**Set epsilons and run model with convergence**

In [ ]:
epsilons= [.005, .005, .005, .005, .005]

In [ ]:
convergence = [EpsilonProgress()]

# convergence = [HyperVolume(minimum=[0,0,0,0,0], maximum=[1e+08, 1e+09, 1e+10, 1000, .1]),
#                EpsilonProgress()]

In [ ]:
with MultiprocessingEvaluator(model) as evaluator:
    output = evaluator.optimize(nfe=nfe, searchover='levers', reference = scenarios[case],
                                              epsilons=epsilons,
                                              convergence=convergence)

**Save files**

In [ ]:
experiment_number = 1

In [ ]:
import pickle
with open(f'./output/directed_search/Directed_Search_{case}_{experiment_number}.kwakkel', 'wb') as filehandler:
    pickle.dump(output, filehandler)
    filehandler.close()

In [ ]:
results, convergence = output

try:
    results.to_csv(f'./output/directed_search/Directed_Search_{case}_{experiment_number}_results.csv', index=False)
    convergence.to_csv(f'./output/directed_search/Directed_Search_{case}_{experiment_number}_convergence.csv', index=False)
except:
    print("Unable to save as csv files")

**Define outcomes, generate parcoords plot, plot epsilon convergence**

In [ ]:
outcomes = results.loc[:,['Expected Annual Damage','Dike Investment Costs','RfR Investment Costs','Evacuation Costs','Expected Number of Deaths']]

In [ ]:
from ema_workbench.analysis import parcoords
import matplotlib.pyplot as plt

limits = parcoords.get_limits(outcomes)
axes = parcoords.ParallelAxes(limits)
axes.plot(outcomes)

plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots(ncols=1, sharex=True, figsize=(8,4))
ax1.plot(convergence.nfe, convergence.epsilon_progress)
ax1.set_ylabel('$\epsilon$-progress')

ax1.set_xlabel('number of function evaluations')
plt.show()

**Load files, import key packages, define model**

In [ ]:
import pickle
with open(f'../final assignment/output/directed_search/Directed_Search_{case}_{experiment_number}.kwakkel', 'rb') as filehandler:
    output= pickle.load(filehandler)
    filehandler.close()

In [ ]:
from ema_workbench import (ScalarOutcome, Scenario, MultiprocessingEvaluator, SequentialEvaluator)
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress)
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.util import ema_logging
import pandas as pd
import numpy as np
ema_logging.log_to_stderr(ema_logging.INFO)

model, planning_steps = get_model_for_problem_formulation(pf_id)

In [ ]:
results, convergence = output

**Robustness: Regret**

In [ ]:
policies = results
policies = policies.drop([o.name for o in model.outcomes], axis=1)

In [ ]:
max_regret = pd.DataFrame(max_regret)
sns.heatmap(max_regret/max_regret.max(), cmap='viridis', annot=True)
plt.show()

In [ ]:
colors = sns.color_palette()

data = max_regret

limits = parcoords.get_limits(data)

paraxes = parcoords.ParallelAxes(limits)
for i, (index, row) in enumerate(data.iterrows()):
    paraxes.plot(row.to_frame().T, label=str(index), color=colors[i])
paraxes.legend()

plt.show()

In [ ]:
from collections import defaultdict

policy_regret = defaultdict(dict)
for key, value in overall_regret.items():
    for policy in value:
        policy_regret[policy][key] = value[policy]

In [ ]:
# this generates a 2 by 2 axes grid, with a shared X and Y axis
# accross all plots
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(10,10),
                         sharey=True, sharex=True)

# to ensure easy iteration over the axes grid, we turn it
# into a list. Because there are four plots, I hard coded
# this.
axes = [axes[0,0], axes[0,1],
        axes[1,0],]

# zip allows us to zip together the list of axes and the list of
# key value pairs return by items. If we iterate over this
# it returns a tuple of length 2. The first item is the ax
# the second items is the key value pair.
for ax, (policy, regret) in zip(axes, policy_regret.items()):
    data = pd.DataFrame(regret)

    # we need to scale the regret to ensure fair visual
    # comparison. We can do that by divding by the maximum regret
    data = data/max_regret.max(axis=0)
    sns.boxplot(data=data, ax=ax)

    # removes top and left hand black outline of axes
    sns.despine()

    # ensure we know which policy the figure is for
    ax.set_title(str(policy))
plt.show()

**Needs to be Fixed**

the choice for box 42 is somewhat arbitrary.